##  Activation likelihood estimation
This code is largely based on the implementation provided by Enge et al. (2021), available at https://osf.io/34ry2/. We are deeply grateful for their dedication to open research.

> Enge, A., Abdel Rahman, R., & Skeide, M. A. (2021). A meta-analysis of fMRI studies of semantic cognition in children. NeuroImage, 241, 118436. https://doi.org/10.1016/j.neuroimage.2021.118436

In [1]:
# Import necessary modules
from os import makedirs, path
import numpy as np
from IPython.display import display
from nibabel import save
from nilearn import glm, image, plotting, reporting
from nimare import io, meta


## ALE analysis: Conjunction analysis

In [2]:
# Before doing the actual subtraction analyses, let's define a helper function for statistical thresholding. Since no FWE correction method has been defined for subtraction analyses (yet), we use an uncorrected voxel-level threshold (usually $p<.001$) combined with a cluster-level extent threshold (in mm<sup>3</sup>). Note that we assume the voxel size to be 2×2×2 mm<sup>3</sup> (the default in NiMARE).

# Define helper function for dual threshold based on voxel-p and cluster size (in mm3)
def dual_thresholding(
    img_z, voxel_thresh, cluster_size_mm3, two_sided=True, fname_out=None
):

    # If img_z is a file path, we first need to load the actual image
    img_z = image.load_img(img=img_z)

    # Check if the image is empty
    if np.all(img_z.get_fdata() == 0):
        print("THE IMAGE IS EMPTY! RETURNING THE ORIGINAL IMAGE.")
        return img_z

    # Convert desired cluster size to the corresponding number of voxels
    k = cluster_size_mm3 // 8

    # Actual thresholding
    img_z_thresh, thresh_z = glm.threshold_stats_img(
        stat_img=img_z,
        alpha=voxel_thresh,
        height_control="fpr",
        cluster_threshold=k,
        two_sided=two_sided,
    )

    # Print the thresholds that we've used
    print(
        "THRESHOLDING IMAGE AT Z > "
        + str(thresh_z)
        + " (P = "
        + str(voxel_thresh)
        + ") AND K > "
        + str(k)
        + " ("
        + str(cluster_size_mm3)
        + " mm3)"
    )

    # If requested, save the thresholded map
    if fname_out:
        save(img_z_thresh, filename=fname_out)

    return img_z_thresh


In [3]:
# Now we can go on to perform the actual subtraction analyses. We again define a helper function for this so we can apply this to multiple Sleuth files with a single call (and also reuse it in later notebooks). We simply read two Sleuth files into NiMARE and let its `meta.cbma.ALESubtraction()` function do the rest (as briefly described above). It outputs an unthresholded *z* score map which we then threshold using our helper function.
# Define function for performing a single ALE subtraction analysis
def run_subtraction(
    text_file1,
    text_file2,
    voxel_thresh,
    cluster_size_mm3,
    random_seed,
    n_iters,
    output_dir,
):

    # Let's show the user what we are doing
    print(
        'SUBTRACTION ANALYSIS FOR "'
        + text_file1
        + '" VS. "'
        + text_file2
        + '" WITH '
        + str(n_iters)
        + " PERMUTATIONS"
    )

    # Set a random seed to make the results reproducible
    if random_seed:
        np.random.seed(random_seed)

    # Read Sleuth files
    dset1 = io.convert_sleuth_to_dataset(text_file=text_file1)
    dset2 = io.convert_sleuth_to_dataset(text_file=text_file2)

    # Actually perform subtraction analysis
    sub = meta.cbma.ALESubtraction(n_iters=n_iters, low_memory=False)
    sres = sub.fit(dset1, dset2)

    # Save the unthresholded z map
    img_z = sres.get_map("z_desc-group1MinusGroup2")
    makedirs(output_dir, exist_ok=True)
    name1 = path.basename(text_file1).replace(".txt", "")
    name2 = path.basename(text_file2).replace(".txt", "")
    prefix = output_dir + "/" + name1 + "_minus_" + name2
    save(img_z, filename=prefix + "_z.nii.gz")

    # Create and save the thresholded z map
    dual_thresholding(
        img_z=img_z,
        voxel_thresh=voxel_thresh,
        cluster_size_mm3=cluster_size_mm3,
        two_sided=True,
        fname_out=prefix + "_z_thresh.nii.gz",
    )



## ALE analysis: Contrast analysis

In [4]:
# Define helper function for dual threshold based on voxel-p and cluster size (in mm3)
def dual_thresholding(
    img_z, voxel_thresh, cluster_size_mm3, two_sided=True, fname_out=None
):

    # If img_z is a file path, we first need to load the actual image
    img_z = image.load_img(img=img_z)

    # Check if the image is empty
    if np.all(img_z.get_fdata() == 0):
        print("THE IMAGE IS EMPTY! RETURNING THE ORIGINAL IMAGE.")
        return img_z

    # Convert desired cluster size to the corresponding number of voxels
    k = cluster_size_mm3 // 8

    # Actual thresholding
    img_z_thresh, thresh_z = glm.threshold_stats_img(
        stat_img=img_z,
        alpha=voxel_thresh,
        height_control="fpr",
        cluster_threshold=k,
        two_sided=two_sided,
    )

    # Print the thresholds that we've used
    print(
        "THRESHOLDING IMAGE AT Z > "
        + str(thresh_z)
        + " (P = "
        + str(voxel_thresh)
        + ") AND K > "
        + str(k)
        + " ("
        + str(cluster_size_mm3)
        + " mm3)"
    )

    # If requested, save the thresholded map
    if fname_out:
        save(img_z_thresh, filename=fname_out)

    return img_z_thresh

In [5]:
# Define function for performing a single ALE subtraction analysis
def run_subtraction(
    text_file1,
    text_file2,
    voxel_thresh,
    cluster_size_mm3,
    random_seed,
    n_iters,
    output_dir,
):

    # Let's show the user what we are doing
    print(
        'SUBTRACTION ANALYSIS FOR "'
        + text_file1
        + '" VS. "'
        + text_file2
        + '" WITH '
        + str(n_iters)
        + " PERMUTATIONS"
    )

    # Set a random seed to make the results reproducible
    if random_seed:
        np.random.seed(random_seed)

    # Read Sleuth files
    dset1 = io.convert_sleuth_to_dataset(text_file=text_file1)
    dset2 = io.convert_sleuth_to_dataset(text_file=text_file2)

    # Actually perform subtraction analysis
    sub = meta.cbma.ALESubtraction(n_iters=n_iters, low_memory=False)
    sres = sub.fit(dset1, dset2)

    # Save the unthresholded z map
    img_z = sres.get_map("z_desc-group1MinusGroup2")
    makedirs(output_dir, exist_ok=True)
    name1 = path.basename(text_file1).replace(".txt", "")
    name2 = path.basename(text_file2).replace(".txt", "")
    prefix = output_dir + "/" + name1 + "_minus_" + name2
    save(img_z, filename=prefix + "_z.nii.gz")

    # Create and save the thresholded z map
    dual_thresholding(
        img_z=img_z,
        voxel_thresh=voxel_thresh,
        cluster_size_mm3=cluster_size_mm3,
        two_sided=True,
        fname_out=prefix + "_z_thresh.nii.gz",
    )

In [6]:
text_file1 ="../data/ale_analysis_foci/control.txt"
text_file2 ="../data/ale_analysis_foci/patient.txt"

output_dir="../results/subtraction"

In [7]:
run_subtraction(
    text_file1=text_file1,
    text_file2=text_file2,
    voxel_thresh=0.001,
    cluster_size_mm3=200,
    random_seed=1234,
    n_iters=10,
    output_dir="../results/subtraction"
    )

SUBTRACTION ANALYSIS FOR "../data/ale_analysis_foci/control.txt" VS. "../data/ale_analysis_foci/patient.txt" WITH 10 PERMUTATIONS


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/228483 [00:00<?, ?it/s]

THRESHOLDING IMAGE AT Z > 3.2905267314918945 (P = 0.001) AND K > 25 (200 mm3)


/opt/anaconda3/lib/python3.12/site-packages/nilearn/glm/thresholding.py:297: UserWarning: The given float value must not exceed 1.6448536269514729. But, you have given threshold=3.2905267314918945.
  stat_img = threshold_img(


In [ ]:
# Compute seperate difference maps for children > adults and adults > children
img_sub = image.load_img(output_dir + "children_minus_adults_z_thresh.nii.gz")
img_children_gt_adults = image.math_img("np.where(img > 0, img, 0)", img=img_sub)
img_adults_gt_children = image.math_img("np.where(img < 0, img * -1, 0)", img=img_sub)
_ = save(img_children_gt_adults, output_dir + "children_greater_adults_z_thresh.nii.gz")
_ = save(img_adults_gt_children, output_dir + "adults_greater_children_z_thresh.nii.gz")


In [ ]:

# Finally, we also compute a conjunction map. This map shows all the brain regions that are engaged in semantic cognition in *both* groups (but not those specific to either one of them). For these voxels, we just take the smaller of the two *z* values from both group-specific *z* score maps (Nichols et al., 2005, *NeuroImage*). We then do the same for the ALE maps so we have our conjunction maps with both *z* scores and ALE values.


# Compute conjunction z map (= minimum voxel-wise z score across both groups)
formula = "np.where(img1 * img2 > 0, np.minimum(img1, img2), 0)"
img_adults_z = image.load_img(output_dir + "adults_z_thresh.nii.gz")
img_children_z = image.load_img("../results/ale/all_z_thresh.nii.gz")
img_conj_z = image.math_img(formula, img1=img_adults_z, img2=img_children_z)
_ = save(img_conj_z, output_dir + "children_conj_adults_z.nii.gz")

# Compute conjunction ALE map (= minimum voxel-wise ALE value across both groups)
img_adults_ale = image.load_img(output_dir + "adults_stat_thresh.nii.gz")
img_children_ale = image.load_img("../results/ale/all_stat_thresh.nii.gz")
img_conj_ale = image.math_img(formula, img1=img_adults_ale, img2=img_children_ale)
_ = save(img_conj_ale, output_dir + "children_conj_adults_ale.nii.gz")